In [1]:
from ARMTypes import *
from ARMAST import *
from asl.ARMParser import get_parser
from asl.ARMAST import Instruction
import json
print("loading...")
with open("intr.json", 'r') as fi:
    I = json.load(fi)
with open("asl/arm_instrs.sexpr") as f:
    data = f.read()
parser = get_parser()
print("parsing...")
Instrs: List[Instruction] = parser.parse(data)
print("parse done...")
from importlib import reload


loading...
parsing...
parse done...


In [2]:
import ARMIntrinsics
reload(ARMIntrinsics)

Skipped encodings: ['LD1_asisdlse_R1_1v', 'ST1_asisdlse_R1_1v', 'SSHR_asimdshf_R', 'SSHR_asisdshf_R', 'USHR_asimdshf_R', 'USHR_asisdshf_R', 'SHL_asimdshf_R', 'SHL_asisdshf_R', 'SRSHR_asimdshf_R', 'SRSHR_asisdshf_R', 'URSHR_asimdshf_R', 'URSHR_asisdshf_R', 'SSRA_asimdshf_R', 'SSRA_asisdshf_R', 'USRA_asimdshf_R', 'USRA_asisdshf_R', 'SRSRA_asimdshf_R', 'SRSRA_asisdshf_R', 'URSRA_asimdshf_R', 'URSRA_asisdshf_R', 'SQSHLU_asimdshf_R', 'SQSHLU_asisdshf_R', 'SHRN_asimdshf_N', 'SQSHRUN_asimdshf_N', 'SQSHRUN_asisdshf_N', 'SQRSHRUN_asimdshf_N', 'SQRSHRUN_asisdshf_N', 'SQSHRN_asimdshf_N', 'SQSHRN_asisdshf_N', 'UQSHRN_asimdshf_N', 'UQSHRN_asisdshf_N', 'RSHRN_asimdshf_N', 'SQRSHRN_asimdshf_N', 'SQRSHRN_asisdshf_N', 'UQRSHRN_asimdshf_N', 'UQRSHRN_asisdshf_N', 'SSHLL_asimdshf_L', 'USHLL_asimdshf_L', 'SRI_asimdshf_R', 'SRI_asisdshf_R', 'SLI_asimdshf_R', 'SLI_asisdshf_R', 'INS_asimdins_IR_r', 'DUP_asimdins_DR_r', 'DUP_asimdins_DV_v', 'EXT_asimdext_only', 'UMOV_asimdins_X_x', 'SMOV_asimdins_X_x', 'XAR_VV

<module 'ARMIntrinsics' from '/home/scarlet/Hydride/codegen-generator/targets/arm/ARMIntrinsics.py'>

In [19]:
import ARMSemanticGen
import ARMAST
reload(ARMAST)
reload(ARMSemanticGen)
zzzz = set(ARMIntrinsics.Intrinsics2Encodings.values())
map2AST = {}
for i in Instrs:
    for j in i.instEncodings:
        if j.encName in zzzz:
            wholeCode = j.encDecode
            if type(j.encDecode) != asl.Nothing:
                wholeCode += i.instPostDecode
            map2AST[j.encName] = (
                i.instExecute, wholeCode)
assert len(map2AST) == len(zzzz)
print("map2AST done...")


def getSemaByInstrDesc(intrin):
    # if intrin.name not in ARMIntrinsics.Intrinsics2Encodings:
    #     return None
    if intrin.name not in ARMIntrinsics.Intrinsics2Fields:
        return None
    if any(i in intrin.name for i in ["ld2", "st2", "ld3", "st3", "ld4", "st4"]):
        return None
    # print(intrin)
    # assert False
    inst = intrin.base_instruction[0]
    field = ARMIntrinsics.Intrinsics2Fields[intrin.name]
    enc = ARMIntrinsics.Intrinsics2Encodings[intrin.name]
    ASTs = map2AST[enc]
    D = ARMSemanticGen.DecodeContext(field)

    resolving = D.walk(ARMAST.ASTShrink(ASTs[1]))
    if D.undefined or D.symbolnotfound:
        print(D.undefined, D.symbolnotfound)
        print(intrin.name)
        print(enc, field)
        assert False

    Params, signedness, preparation = ARMSemanticGen.parse_instr_attr(
        intrin)
    for k in preparation:
        del resolving[k]
    return ARMSema(intrin.name,
                   inst,
                   Params,
                   ARMAST.ASTShrink(ASTs[0]),
                   intrin.return_type["value"],
                   signedness,
                   f"{inst} {intrin.operands}",
                   None, None, None, None,
                   preparation,
                   resolving,
                   )


def getSemaByName(name):
    for i in I:
        if i["name"] == name:
            return getSemaByInstrDesc(InstrDesc(**i))
    return None


def genAllSema():
    for i in I:
        yield getSemaByInstrDesc(InstrDesc(**i))


# getSemaByName("vadd_s8")
allsema = [i for i in genAllSema() if i is not None]
print(len(allsema))


map2AST done...
1154
